In [6]:
import pandas as pd
import time

# 数据加载
data = pd.read_csv('./BreadBasket_DMS.csv')
# 统一小写
data['Item'] = data['Item'].str.lower()
# 去掉none项
data = data.drop(data[data.Item == 'none'].index)

# 采用efficient_apriori工具包
def rule1():
    from efficient_apriori import apriori
    start = time.time()
    # 得到一维数组orders_series，并且将Transaction作为index, value为Item取值
    orders_series = data.set_index('Transaction')['Item']
    # 将数据集进行格式转换
    # transaction为订单数组
    transactions = []
    temp_index = 0
    for i, v in orders_series.items():
        if i != temp_index:
            temp_set = set()
            temp_index = i
            temp_set.add(v)
            transactions.append(temp_set)
        else:
            temp_set.add(v)

    # 挖掘频繁项集和频繁规则
    # 去尝试min_support & min_confidence
    itemsets, rules = apriori(transactions, min_support=0.02,  min_confidence=0.5)
    print('频繁项集：', itemsets)
    print('关联规则：', rules)
    end = time.time()
    print("用时：", end-start)


def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
# 采用mlxtend.frequent_patterns工具包
def rule2():
	from mlxtend.frequent_patterns import apriori
	from mlxtend.frequent_patterns import association_rules
	pd.options.display.max_columns=100
	start = time.time()
	hot_encoded_df=data.groupby(['Transaction','Item'])['Item'].count().unstack().reset_index().fillna(0).set_index('Transaction')
	hot_encoded_df = hot_encoded_df.applymap(encode_units)
     # set support
	frequent_itemsets = apriori(hot_encoded_df, min_support=0.02, use_colnames=True)
    # set lifts
	rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
    
	print("频繁项集：", frequent_itemsets)
	print("关联规则：", rules[ (rules['lift'] >= 1) & (rules['confidence'] >= 0.5) ])
	#print(rules['confidence'])
	end = time.time()
	print("用时：", end-start)

rule1()
print('-'*100)
rule2()




----------------------------------------------------------------------------------------------------


ModuleNotFoundError: No module named 'mlxtend'

In [5]:
import pandas as pd
import time

# 数据加载
data = pd.read_csv('./BreadBasket_DMS.csv')
# 统一小写
data['Item'] = data['Item'].str.lower()
# 去掉none项
data = data.drop(data[data.Item == 'none'].index)
orders_series = data.set_index('Transaction')['Item']
print(orders_series)

Transaction
2        scandinavian
2        scandinavian
3       hot chocolate
3                 jam
3             cookies
            ...      
9682           coffee
9682              tea
9683           coffee
9683           pastry
9684        smoothies
Name: Item, Length: 20506, dtype: object
